# GRANDE: GRADIENT-BASED DECISION TREE ENSEMBLES FOR TABULAR DATA

## Abstract
Tree-based ensemble models maintain state-of-the-art performance for heterogeneous tabular data despite deep learning advances in other domains. This paper presents GRANDE (GRAdient-Based Decision Tree Ensembles), a method for learning hard, axis-aligned decision tree ensembles through end-to-end gradient descent. The approach employs a dense representation of tree ensembles and utilizes backpropagation with a straight-through operator for joint optimization of all model parameters. By combining axis-aligned splits with gradient-based optimization, GRANDE preserves the advantageous inductive bias of tree-based methods while gaining the flexibility of gradient descent. An advanced instance-wise weighting mechanism enables the model to learn representations for both simple and complex relations within a single ensemble. Extensive evaluation on a predefined benchmark comprising 19 classification datasets demonstrates that GRANDE outperforms existing gradient-boosting and deep learning frameworks on most datasets.

## Introduction
Heterogeneous tabular data represents the most frequently utilized data form across diverse applications including medical diagnosis, creditworthiness estimation, and fraud detection. Despite deep learning achievements in various domains, recent studies indicate that tree-based models such as XGBoost and CatBoost continue to outperform deep learning methods for tabular data in most cases. However, gradient-based training offers substantial advantages including high flexibility through easy integration of arbitrary differentiable loss functions, support for iterative training, and straightforward incorporation into multimodal learning scenarios where tabular data constitutes one of several input types.

Building upon GradTree, which introduced gradient descent for learning hard, axis-aligned decision trees, this work extends the approach to tree ensembles. GRANDE maintains the important advantage that hard, axis-aligned decision trees learn piece-wise constant functions without bias toward smooth solutions, a characteristic where deep learning methods typically struggle with tabular datasets. This represents a fundamental distinction from existing deep learning methods for hierarchical representations such as NODE, which employs soft and oblique splits.

The primary contributions include extension of GradTree to an end-to-end gradient-based tree ensemble with efficient computation, introduction of softsign as a differentiable split function, and proposal of a novel instance-wise weighting technique that emphasizes estimator importance on a per-instance basis. Comprehensive evaluation on 19 binary classification tasks demonstrates that GRANDE achieves superior performance for both default and optimized hyperparameters, with substantial performance differences on several datasets.

## Background: Gradient-Based Decision Trees
GRANDE builds upon gradient-based decision trees (GradTree) at the individual tree level. Traditional decision trees involve nested concatenation of rules, but GradTree reformulates decision trees as arithmetic functions based on addition and multiplication to enable gradient-based learning. A decision tree of depth d is formulated as $$t(x \mid \lambda, \tau, \iota) = \sum_{l=0}^{2^d - 1} \lambda_l \, \mathcal{L}(x \mid l, \tau, \iota)$$, where $\mathcal{L}$ indicates whether sample $x \in \mathbb{R}^n$ belongs to leaf $l, \lambda \in \mathbb{C}^{2^d}$ denotes class membership for each leaf node, $\tau \in \mathbb{R}^{2^{d-1}}$ represents split thresholds, and $\iota \in \mathbb{N}^{2^{d-1}}$ represents feature indices for each internal node.
GradTree introduces a dense decision tree representation to support gradient-based optimization and efficient computation via matrix operations. The feature index vector is expanded from one-dimensional form into matrix representation through one-hot encoding, converting $\iota \in \mathbb{R}^{2^{d-1}}$ into $I \in \mathbb{R}^{(2^{d-1}) \times n}$. Similarly, split thresholds are expanded to store individual values for each feature, yielding $T \in \mathbb{R}^{(2^{d-1}) \times n}$. This dense representation enables redefinition of the decision tree as $$g(x \mid \lambda, T, I) = \sum_{l=0}^{2^d - 1} \lambda_l \, \mathcal{L}(x \mid l, T, I)$$.
The split function addresses the non-differentiable nature of the Heaviside step function through reformulation: $$S(x \mid \iota, \tau) = \lfloor \mathcal{S}(\iota \cdot x - \iota \cdot \tau) \rceil$$, where $\mathcal{S}(z) = \frac{1}{1 + e^{-z}}$ represents the logistic function and $\lfloor z \rceil$ denotes rounding. The straight-through operator enables use of non-differentiable operations in the forward pass while ensuring gradient propagation in the backward pass.



## GRANDE: Gradient-Based Decision Tree Ensembles
#### From Decision Trees to Weighted Tree Ensembles
The extension from individual trees to weighted ensembles represents a core contribution. GRANDE maintains the inductive bias of axis-aligned splits for tabular data while combining this property with end-to-end gradient-based optimization. The ensemble is defined as
$$G(x \mid \omega, \mathcal{L}, T, I) = \sum_{e=0}^E \omega_e \, g(x \mid \mathcal{L}_e, T_e, I_e)$$,
where $E$ represents the number of estimators and $\omega$ is a weight vector. Extension of matrices and tensors for the complete ensemble enables leveraging parallel computation for efficient training.

End-to-end gradient descent training provides important advantages over non-gradient-based tree methods. Both the sequential induction of individual trees and their sequential combination via boosting are greedy processes in methods like XGBoost and CatBoost, resulting in search space constraints and potential overfitting. GRANDE learns all ensemble parameters jointly, overcoming these limitations.


#### Differentiable Split Functions

![Description](Differentiable_Split_Functions.png)


Various differentiable split functions have been proposed as alternatives to the non-differentiable Heaviside step function. Prevalent approaches include sigmoid functions for soft decisions and entmax transformations for sparse decisions. The softsign function, scaled to the range (0, 1), is proposed as a differentiable split function:
$$S_{ss}(z) = \frac{1}{2} \left( \frac{z}{1 + |z|} + 1 \right)$$

The softsign gradient characteristics exhibit pronounced behavior when samples are close to thresholds, reduce sharply, yet maintain responsive gradients when substantial differences exist between feature values and thresholds. These characteristics provide superiority for differentiable splitting compared to sigmoid functions with smooth gradient decline and entmoid functions with gradients that drop to zero for large value differences


#### Instance-Wise Estimator Weights
Learning appropriate weighting schemes for individual estimators represents a significant challenge in ensemble methods. Simple solutions employing one weight per estimator with softmax normalization force homogeneous ensembles where each tree aims for equally good predictions across all samples. A more beneficial approach allows individual trees to account for different target function areas without requiring confident predictions for every sample.

An advanced weighting scheme calculates instance-wise weights learned within gradient-based optimization. Rather than using one weight per estimator, one weight is assigned for each leaf of each estimator, defining weights as $W \in \mathbb{R}^{E \times 2^d}$ instead of $\omega \in \mathbb{R}^E$. The function $w(x \mid W, \mathcal{L}, T, I): \mathbb{R}^n \rightarrow \mathbb{R}^E$ calculates a weight vector with one weight per tree based on the leaf to which the current sample is assigned. Softmax is applied to these weights, and multiplication by predicted values from each tree computes a weighted average:

$$G(x \mid W, \mathcal{L}, T, I) = \sigma(w(x \mid W, \mathcal{L}, T, I)) \cdot p(x \mid \mathcal{L}, T, I)$$
This weighting scheme permits calculation of instance-wise weights for unseen samples and differs substantially from existing tree ensemble methods and post-hoc weighting schemes by incorporating into the training procedure to capture local interactions.

![Description](Grande_Architecture.png)


#### Regularization: Feature Subset, Data Subset and Dropout
The combination of tree-based methods with gradient-based optimization enables application of numerous regularization techniques. Feature subset selection for each tree regularizes the model while addressing poor scalability of GradTree with increasing feature numbers. Sample subset selection for each estimator and dropout through random deactivation of predefined estimator fractions with weight rescaling provide additional regularization mechanisms.

## Experimental Evaluation
#### Experimental Setup
The evaluation employed a predefined collection of 19 binary classification datasets selected based on objective criteria from OpenML Benchmark Suites. The selection process, adopted from established benchmarks, avoids bias toward the proposed method. Preprocessing included one-hot encoding for low-cardinality categorical features, leave-one-out encoding for high-cardinality categorical features (exceeding 10 categories), and gaussianization via quantile transformation. Mean and standard deviation of test performance over 5-fold cross-validation were reported to ensure reliability.

The evaluation compared GRANDE to XGBoost and CatBoost as state-of-the-art tree-based methods and NODE as the most related gradient-based approach, providing one representative method for each tree type (standard and oblivious decision trees) in both tree-based and gradient-based categories. Hyperparameters were optimized using Optuna with 250 trials, with best parameters selected based on 5×2 cross-validation and test data held out from hyperparameter optimization to obtain unbiased results. Class weights addressed class imbalance.

Table 2: Performance Comparison. We report the test macro F1-score (mean ± stdev for a 5-fold
CV) with optimized parameters. The datasets are sorted based on the data size.
![Description](results_hpo.jpg)

#### Results
GRANDE outperformed existing methods on most datasets with optimized hyperparameters based on macro F1-Score, achieving the highest mean reciprocal rank of 0.702 and highest normalized mean of 0.776. CatBoost yielded second-best results (MRR of 0.570 and normalized mean of 0.671) followed by XGBoost (MRR of 0.417 and normalized mean of 0.483) and NODE (MRR of 0.395 and normalized mean of 0.327). Findings align with existing work indicating no universal method for tabular data exists. However, substantial performance differences on several datasets such as climate-simulation-crashes and cylinder-bands highlight GRANDE's importance as an extension to existing methods. Results were particularly strong for small datasets.


\begin{array}{lccccc}
\textbf{Metric} & \text{Softsign} & \text{Entmoid} & \text{Sigmoid} & \text{Leaf Weights} & \text{Estimator Weights} \\
\hline
\text{Normalized Mean } \uparrow & 0.7906\ (1) & 0.4188\ (2) & 0.2207\ (3) & 0.8235\ (1) & 0.1765\ (2) \\
\text{MRR } \uparrow & 0.8246\ (1) & 0.5526\ (2) & 0.4561\ (3) & 0.9211\ (1) & 0.5789\ (2) \\
\end{array}


GRANDE demonstrated efficiency for large and high-dimensional datasets, averaging 47 seconds across all datasets with maximum runtime of 107 seconds. Runtime remained robust for high-dimensional datasets (37 seconds for Bioresponse with 1,776 features) and larger datasets (39 seconds for numerai28.6 with 96,320 samples), achieving significantly lower runtime compared to NODE which averaged approximately 130 seconds.

GRANDE achieved superior results with default hyperparameters, significantly outperforming existing methods on most datasets with the highest normalized mean performance (0.6371) and highest MRR (0.6404). Ablation studies confirmed that softsign as split activation function provided superior performance compared to sigmoid and entmoid alternatives, and instance-wise weighting based on leaf weights substantially increased model performance compared to single weights per estimator.

\begin{array}{lcc}
\textbf{Method} & \text{Normalized Mean } \uparrow & \text{Mean Reciprocal Rank (MRR) } \uparrow \\
\hline
\text{GRANDE} & $0.6371\ (1)$ & $0.6404\ (1)$ \\
\text{XGB} & $0.5865\ (2)$ & $0.5175\ (3)$ \\
\text{CatBoost} & $0.5793\ (3)$ & $0.5219\ (2)$ \\
\text{NODE} & $0.2698\ (4)$ & $0.4035\ (4)$ \\
\end{array}

#### Case Study: Instance-Wise Weighting for the PhishingWebsites Dataset
The $PhishingWebsites$ dataset demonstrates GRANDE's ability to learn compact representations for simple rules within complex ensembles. Although the task is challenging, several clear indicators for phishing websites exist. For instances easily categorized using simple rules, the model should follow these rules for predictions. One universal rule in the dataset identifies instances as phishing if a prefix or suffix was added to the domain name.

GRANDE learns a simple, transparent rule in the dataset: if a domain has a prefix or suffix added, classify it as phishing. For an example case, the decision tree (Figure 3) contributes 94% of the prediction, showing clear estimator importance. Competing ensemble methods (e.g., XGBoost, CatBoost) don’t offer such out-of-the-box estimator interpretability because they sum predictions sequentially or weight all estimators equally. This interpretability and tailored weighting improve GRANDE’s average performance over using a single weight per estimator.

Figure 3: Highest-Weighted Estimator.
This figure visualizes the DT from GRANDE (1024 total estimators) which has the highest weight for an exemplary instance.

![Description](Highest_Weighted_Estimator.png)


Instance-wise weighting demonstrates notable impact on local interpretability. For each instance, weights of individual estimators can be assessed and highest-importance estimators inspected to understand which rules most impact predictions. For the given example (Figure 3), only a single tree of depth two required observation to understand the phishing classification, despite overall model complexity. Existing ensemble methods require global interpretation without providing simple local explanations out-of-the-box.

Comparison with Anchors, which provides model-agnostic explanations through identifying conditions guaranteeing predictions with high probability, revealed that the anchor extracted for GRANDE comprised a single rule with precision of 1.00 and significantly higher coverage compared to other methods. This indicates the rule learned by GRANDE is more broadly representative, while explanations for other approaches exhibited significantly higher complexity with multiple rules required.

## Related Work
Existing methods for tabular data can be divided into tree-based, deep learning, and hybrid categories. 

**Tree-based methods**, particularly gradient-boosted decision trees, have been widely used due to their ability to capture non-linear relationships. Prominent GBDT methods include XGBoost with advanced regularization, CatBoost with special handling for categorical variables, and LightGBM with leaf-wise growth strategy. 

**Deep learning** methods have adapted architectures, primarily transformers, to tabular data, with SAINT representing the superior deep learning method using attention over both rows and columns.

**Hybrid methods** aim to combine gradient-based optimization strengths with other algorithms, commonly tree-based methods. One prominent approach uses soft decision trees applying gradient descent by replacing hard decisions with soft ones and axis-aligned with oblique splits. NODE learns ensembles of oblivious decision trees with gradient descent and is closely related to this work. Oblivious decision trees use identical splitting features and thresholds for internal nodes at the same depth, enabling efficient parallel computation suitable for weak learners. GRANDE uses standard decision trees as weak learners.

Recent studies indicate that despite substantial effort in developing high-performing deep learning methods, tree-based models still outperform deep learning for tabular data, though the gap is diminishing. One primary reason for superior tree-based method performance lies in axis-aligned splits not biased toward overly smooth solutions. GRANDE aligns with this argument, using hard axis-aligned splits combined with gradient-based optimization flexibility.

## Conclusion and Future Work
GRANDE introduces a method for learning hard, axis-aligned tree ensembles with gradient descent, combining the advantageous inductive bias of axis-aligned splits with gradient descent optimization flexibility. Extensive evaluation on a predefined benchmark demonstrated that GRANDE achieved superior results with both optimized and default parameters, outperforming existing state-of-the-art methods on most datasets. Instance-wise weighting emphasizes learning representations for simple and complex relations within a single model, increasing local interpretability compared to existing methods.
The current architecture is a shallow ensemble already achieving state-of-the-art performance. However, gradient-based optimization flexibility holds potential for future extensions including categorical embeddings, stacking of tree layers, and incorporation of tree layers into deep learning frameworks.

## References
https://arxiv.org/pdf/2309.17130
